In [14]:
from IPython.core.magic import Magics, cell_magic, line_magic, magics_class
from pexpect import spawn

TIMEOUT = 300
PROG = "hive"
PROMPT = ["\r\n    > ", "\r\nhive> "]
QUIT = "quit;"


@magics_class
class Magic(Magics):
    def __init__(self, shell):
        super().__init__(shell)
        self.app = spawn(PROG, timeout=60)
        self.app.expect(PROMPT)

    @cell_magic
    def hive(self, line, cell):
        cell_lines = [cell_line.strip() for cell_line in cell.split("\n")]
        cell_lines = [cell_line for cell_line in cell_lines if cell_line != ""]
        for cell_line in cell_lines:
            self.app.sendline(cell_line)
            self.app.expect(PROMPT, timeout=TIMEOUT)
            output = self.app.before.decode()
            output = output.replace("\r\n", "\n")
            output = output.split("\n")
            output = [output_line.strip() for output_line in output]
            for output_line in output:
                if output_line not in cell_lines:
                    print(output_line)
        return None

    @line_magic
    def quit(self, line):
        self.app.sendline(QUIT)


def load_ipython_extension(ip):
    ip.register_magics(Magic(ip))


load_ipython_extension(ip=get_ipython())

In [7]:
!hdfs dfs -copyFromLocal pregunta_02/data.tsv /tmp

In [15]:
!hdfs dfs -ls /tmp

Found 3 items
-rw-r--r--   1 root supergroup        677 2023-05-31 02:45 /tmp/data.tsv
drwxrwx---   - root supergroup          0 2023-05-30 16:19 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2023-05-30 16:21 /tmp/hive


In [5]:
!hdfs dfs -rm /tmp/data.tsv

Deleted /tmp/data.tsv


In [16]:
%%hive
DROP TABLE IF EXISTS data;

FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [12]:
%%hive
CREATE TABLE data (
letter STRING,
date_event STRING,
value INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY  '\t';

FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [13]:
%%hive
LOAD DATA INPATH '/tmp/data.tsv' OVERWRITE INTO TABLE data;
SELECT * FROM data;

FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


KeyboardInterrupt: 

In [ ]:
%%hive
SELECT *
FROM data
ORDER BY letter, value;